<a href="https://colab.research.google.com/github/omkarpat/eeg-mood-classification/blob/master/Models/Baseline_Raw_Spectrogram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
# Colab settings/mount
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/My\ Drive/CSE\ 240/Project/Data/spectrogram_raw/

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
[Errno 2] No such file or directory: 'gdrive/My Drive/CSE 240/Project/Data/spectrogram_raw/'
/content/gdrive/My Drive/CSE 240/Project/Data/spectrogram_raw


In [62]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, StratifiedKFold

%tensorflow_version 2.x
import tensorflow
print(tensorflow.__version__)
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Convolution2D, Conv2D, MaxPooling2D, Lambda, GlobalMaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Activation, AveragePooling2D, Concatenate, SeparableConv2D, MaxPooling3D, Conv3D, DepthwiseConv2D
from tensorflow.keras.models import model_from_json

2.1.0


In [0]:
!ls

In [64]:
base_path = "."
concentration_data_array = None
relaxed_data_array = None
print("loading data from .npy files...")
for filename in os.listdir(base_path):
  if "E02" in filename and filename.endswith('.npy') or "E04" in filename and filename.endswith('.npy'):
    print(filename)
    concentration_data_array = np.concatenate((concentration_data_array, np.load(filename))) if concentration_data_array is not None else np.load(filename)
  if "E01" in filename and filename.endswith('.npy') or "E03" in filename and filename.endswith('.npy'):
    print(filename)
    relaxed_data_array = np.concatenate((relaxed_data_array, np.load(filename))) if relaxed_data_array is not None else np.load(filename)
print(concentration_data_array.shape, relaxed_data_array.shape)
print(len(concentration_data_array), len(relaxed_data_array))

loading data from .npy files...
S004E01.npy
S004E02.npy
S004E03.npy
S004E04.npy
S005E01.npy
S005E02.npy
S005E03.npy
S005E04.npy
S006E01.npy
S006E02.npy
S006E03.npy
S006E04.npy
S007E01.npy
S007E02.npy
S007E03.npy
S007E04.npy
S008E01.npy
S008E02.npy
S008E03.npy
S008E04.npy
S009E01.npy
S009E02.npy
S009E03.npy
S009E04.npy
S010E01.npy
S010E02.npy
S010E03.npy
S010E04.npy
S011E01.npy
S011E02.npy
S011E03.npy
S011E04.npy
S012E01.npy
S012E02.npy
S012E03.npy
S012E04.npy
S013E01.npy
S013E02.npy
S013E03.npy
S013E04.npy
S014E01.npy
S014E02.npy
S014E03.npy
S014E04.npy
S015E01.npy
S015E02.npy
S015E03.npy
S015E04.npy
S016E01.npy
S016E02.npy
S016E03.npy
S016E04.npy
S017E01.npy
S017E02.npy
S017E03.npy
S017E04.npy
S018E01.npy
S018E02.npy
S018E03.npy
S018E04.npy
S019E01.npy
S019E02.npy
S019E03.npy
S019E04.npy
S020E01.npy
S020E02.npy
S020E03.npy
S020E04.npy
S021E01.npy
S021E02.npy
S021E03.npy
S021E04.npy
S022E01.npy
S022E02.npy
S022E03.npy
S022E04.npy
S023E01.npy
S023E02.npy
S023E03.npy
S023E04.npy
S024E01.

ValueError: ignored

In [0]:
X = np.concatenate((relaxed_data_array, concentration_data_array))
Y = np.concatenate((np.zeros((len(relaxed_data_array),1)),np.ones((len(concentration_data_array),1))))
print(X.shape, Y.shape)
relaxed_data_array, concentration_data_array = None, None

(4320, 14, 65, 45) (4320, 1)


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=42)
X, Y = None, None
folds = list(StratifiedKFold(n_splits=10, shuffle=True, random_state=1).split(X_train, y_train))

In [0]:
model_name = "4-layer-Convolutions-raw"
def get_model():
  model = Sequential()
  
  model.add(Conv2D(64, kernel_size=3, strides=(1,1), padding='valid', activation='relu', input_shape = (14, 65, 45), data_format='NCHW')) #change the data format as the 14 channels are before the X,Y axes
  
  model.add(Flatten())
  model.add(Dense(100, activation='relu'))
  model.add(Dense(50, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  
  sgd = optimizers.SGD(lr=0.000001, momentum=0.7, decay=1e-6)
  model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [0]:
filepath= model_name + "-weights-improvement-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
model = get_model()
for j, (train_idx, val_idx) in enumerate(folds):
  print('\nFold ',j)
  X_train_cv = X_train[train_idx]
  y_train_cv = y_train[train_idx]
  X_valid_cv = X_train[val_idx]
  y_valid_cv= y_train[val_idx]
  # checkpoint
  model.fit(X_train_cv, y_train_cv, validation_data=(X_valid_cv, y_valid_cv), epochs=10, batch_size=32, shuffle=True)
score = model.evaluate(X_test,y_test)
print("Test set metrics: %s: %.2f%%" % (model.metrics_names[1], score[1]*100))
model_json = model.to_json()
with open(model_name + ".json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(model_name + ".h5")
print("Saved model %s to disk" % (model_name))

ValueError: ignored

In [0]:
# load json and create model
json_file = open(model_name+ '.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(model_name + ".h5")
print("Loaded model from disk")

# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
score = loaded_model.evaluate(X_test,y_test, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

In [0]:
a = np.array([[[1,2,3,4],[11,22,33,44],[111,222,333,444]], [[5,6,7,8],[55,66,77,88],[555,666,777,888]]])
print(a.shape)
print(a)

b = a.flatten()[:12]
for i in range(1, 2):
  start = i * 12
  end = (i + 1) * 12
  b = 

print(a.flatten()[:12].reshape(3,4))
print(a.flatten()[:12].reshape(3,4).shape)

(2, 3, 4)
[[[  1   2   3   4]
  [ 11  22  33  44]
  [111 222 333 444]]

 [[  5   6   7   8]
  [ 55  66  77  88]
  [555 666 777 888]]]
[[  1   2   3   4]
 [ 11  22  33  44]
 [111 222 333 444]]
(3, 4)


In [0]:
#print(np.hstack((X_test[0])).shape)
for item in X_test[0]:
  print(item.flatten().shape)

(2925,)
(2925,)
(2925,)
(2925,)
(2925,)
(2925,)
(2925,)
(2925,)
(2925,)
(2925,)
(2925,)
(2925,)
(2925,)
(2925,)
